In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install langchain sentence-transformers faiss-cpu transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 23.1 MB/s eta 0:00:00
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import faiss
import json
from sentence_transformers import SentenceTransformer
import os

# 1. Prepare Resume Best Practices Data:

if os.path.exists("resume_best_practices_software_data.json"):
    with open("resume_best_practices_software_data.json", "r", encoding="utf-8") as f:
        resume_best_practices = json.load(f)
else:
    resume_best_practices = {
        "Software_Experience": [
            "Quantify software development achievements (e.g., 'Reduced bug reports by 20%').",
            "Highlight experience with specific programming languages, frameworks, and tools relevant to the target role (e.g., Python, Java, React, AWS).",
            "Describe contributions to open-source projects or significant personal projects.",
            "Showcase experience with Agile methodologies (Scrum, Kanban).",
            "Emphasize experience with version control systems (Git).",
            "Detail experience with testing frameworks and practices (unit testing, integration testing).",
            "Showcase experience with CI/CD pipelines."
        ],
        "Software_Skills": [
            "List technical skills prominently, grouping them by category (e.g., Programming Languages, Cloud Technologies, Databases).",
            "Prioritize skills mentioned in the job description.",
            "Include both front-end and back-end technologies if applicable.",
            "Mention any relevant certifications (e.g., AWS Certified Solutions Architect).",
            "Be specific about versions or levels of proficiency (e.g., 'Python 3.7+,' 'AWS Certified Solutions Architect - Associate')."
        ],
        "Software_Projects": [
            "Describe software projects in detail, focusing on the technologies used, the challenges overcome, and the results achieved.",
            "Include links to GitHub repositories or live demos if possible.",
            "Quantify the impact of your projects (e.g., 'Developed a web application that increased user engagement by 30%').",
            "Highlight any leadership roles or contributions to team projects."
        ],
        "Data_Experience": [
            "Quantify data-related accomplishments (e.g., 'Improved model accuracy by 10%').",
            "Highlight experience with data analysis tools and techniques (e.g., Pandas, NumPy, Scikit-learn).",
            "Describe experience with machine learning algorithms and models.",
            "Showcase experience with data visualization tools (e.g., Tableau, Power BI).",
            "Emphasize experience with big data technologies (e.g., Hadoop, Spark) if relevant.",
            "Detail experience with database systems (SQL, NoSQL).",
            "Mention experience with cloud-based data services (e.g., AWS S3, Azure Blob Storage)."
        ],
        "Data_Skills": [
            "List data-related skills prominently, grouping them by category (e.g., Data Analysis, Machine Learning, Big Data).",
            "Prioritize skills mentioned in the job description.",
            "Mention specific tools, libraries, and frameworks (e.g., TensorFlow, PyTorch).",
            "Include experience with data mining, data cleaning, and data preprocessing techniques.",
            "List any relevant certifications (e.g., Google Data Analytics Professional Certificate)."
        ],
        "Data_Projects": [
            "Describe data science or data engineering projects in detail, focusing on the data sources, the methodologies used, and the insights gained.",
            "Include metrics to demonstrate the success of your projects.",
            "Highlight any contributions to data-driven decision-making.",
            "Showcase experience with deploying machine learning models to production."
        ],
        "General": [ # General advice applicable to both software and data roles
            "Tailor your resume to each specific job application.",
            "Use action verbs to start your bullet points.",
            "Focus on the impact of your work.",
            "Keep your resume concise and easy to read.",
            "Proofread carefully for any errors."
        ]
    }

    with open("resume_best_practices_software_data.json", "w", encoding="utf-8") as f:
        json.dump(resume_best_practices, f, indent=4)

# 2. Create Embeddings and FAISS Index:

model = SentenceTransformer('all-mpnet-base-v2')  # Or any other suitable model

embeddings = []
data = []  # Store the best practices and their metadata

for category, practices in resume_best_practices.items():
    for practice in practices:
        embedding = model.encode(practice)
        embeddings.append(embedding)
        data.append({"category": category, "practice": practice})  # Store category for retrieval

embeddings_np = np.array(embeddings).astype('float32')
d = embeddings_np.shape[1]

# Choose appropriate FAISS index. IndexFlatL2 is good for smaller datasets. For larger datasets, use IndexIVFFlat or IndexHNSWFlat.
index = faiss.IndexFlatL2(d)  # L2 distance for similarity search
index.add(embeddings_np)

faiss.write_index(index, "resume_best_practices_index.faiss")

with open("resume_best_practices_data.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=4)

print("FAISS index and metadata saved.")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

FAISS index and metadata saved.


In [4]:
# --- Retrieval (Example) ---
index = faiss.read_index("resume_best_practices_index.faiss")
with open("resume_best_practices_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

query = "How should I describe my work experience?"  # Example query
query_embedding = model.encode(query).astype("float32").reshape(1, -1)
D, I = index.search(query_embedding, k=3)  # Retrieve top 3

retrieved_practices = [data[i]["practice"] for i in I[0]]
print("Retrieved Best Practices:")
for practice in retrieved_practices:
    print(f"- {practice}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved Best Practices:
- Keep your resume concise and easy to read.
- Highlight experience with specific programming languages, frameworks, and tools relevant to the target role (e.g., Python, Java, React, AWS).
- Focus on the impact of your work.
